# Problema 1 Apartado e)

Vamos a resolver el problema mediante la función MixedIntegerLinearProgram(). En primer lugar crearemos una instancia p del problema con valores por defecto puesto que queremos maximizar y una variable x que será no entera y no negativa.

In [3]:
p = MixedIntegerLinearProgram()
x = p.new_variable(integer=False, nonnegative=True)
#A continuación añadimo cada una de las condiciones a x1 y x2 indicadas en el enunciado.
p.add_constraint(6*x[1]+4*x[2]<=24)
p.add_constraint(x[1]+2*x[2]<=6)
p.add_constraint(-1*x[1]+x[2]<=1)
p.add_constraint(x[2]<=2)
#Y marcamos al programa la función objetivo.
p.set_objective(5*x[1]+4*x[2])
p.show()

Maximization:
  5.0 x_0 + 4.0 x_1 

Constraints:
  6.0 x_0 + 4.0 x_1 <= 24.0
  x_0 + 2.0 x_1 <= 6.0
  - x_0 + x_1 <= 1.0
  x_1 <= 2.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)


Como podemos observar el método show() nos ordena los datos y los presenta de forma adecuada. Ya sólo queda usar la función solve() que resolverá nuestro problema y nos permitirá leer los valores dados a cada instancia de x.

In [4]:
sol = p.solve()
for i, v in sorted(p.get_values(x).items()):
     print('x_%s = %s' % (i, v))
print("valor alcanzado:",sol)        

x_1 = 3.0
x_2 = 1.5
valor alcanzado: 21.0


Efectivamente encontramos los mismos valores de maximización para x1 y x2 que cuando realizamos el ejercicio a mano. Igualmente el valor máximo alcanzado por la función es 21.

# Problema 4 Apartado a) y b)

Vamos a resolver el problema mediante SAGE. Para ello, en primer lugar, incluiremos los datos como una tupla de pares relacionando coste y beneficio. Definiremos nuestra función por defecto puesto que vamos a maximizar y haremos que las variables de salida sean binarias:

In [115]:
Datos=[(500,70000), (800,85000), (1000,120000), (2000,200000), (1500,160000), (2500,250000), (3500,400000), (3000,300000)]
p4 = MixedIntegerLinearProgram()
y = p4.new_variable(binary=True)
#Condición de al menos 4 anuncios diarios
p4.add_constraint(y[0]+y[1]+y[2]+y[3]+y[4]+y[5]+y[6]+y[7]>=4)
#La suma de los costres no puede exceder 10000
p4.add_constraint(p4.sum(Datos[j][0]*y[j] for j in range(len(Datos)))<=10000)
#La función que maximizamos es la suma de los valores de benefico (audiencia) en la tupla de datos
p4.set_objective(p4.sum(Datos[j][1]*y[j] for j in range(len(Datos))))
#Resolvemos y presentamos datos
print ('Valor objetivo=', p4.solve())
sol=p4.get_values(y)
for key in sorted(sol.keys()):
    print ('y',key+1,'=',sol[key])

Valor objetivo= 1090000.0
y 1 = 1.0
y 2 = 0.0
y 3 = 1.0
y 4 = 1.0
y 5 = 0.0
y 6 = 0.0
y 7 = 1.0
y 8 = 1.0


Según el resultadfo anterior emitimos en las franjas 1,3,4,7 y 8 y conseguimos una audiencia de 1090000.

Podemos realizar el mismo problema anterior con una nomenclatura más sencilla para problemas de mochila:

In [10]:
from sage.numerical.knapsack import knapsack
knapsack(Datos, max=10000)

[1090000.0,
 [(500, 70000),
  (1000, 120000),
  (2000, 200000),
  (3500, 400000),
  (3000, 300000)]]

Y a continuación vamos a resolver el mismo problema añadiendo las restricciones del apartado b):

In [116]:
Datos=[(500,70000), (800,85000), (1000,120000), (2000,200000), (1500,160000), (2500,250000), (3500,400000), (3000,300000)]
p4 = MixedIntegerLinearProgram()
y = p4.new_variable(binary=True)
p4.add_constraint(y[0]+y[1]+y[2]+y[3]+y[4]+y[5]+y[6]+y[7]>=4)
p4.add_constraint(p4.sum(Datos[j][0]*y[j] for j in range(len(Datos)))<=10000)
#Si se emite un spot en la franja 7, deben quedar excluidas de spot las franjas 1, 4, 5, y 8.
p4.add_constraint(y[6]-3*(1-y[6])<=1-y[0]-y[3]-y[4]-y[7])
#SAGE no acepta los siguientes requisitos a la función porque contienen una multiplicación entre variables.
#Si se emite un spot en alguna de las franjas 6 u 8, se debe emitir un spot tanto en la franja 1 como en la 3.
#p4.add_constraint(y[5]+y[7]<=2*y[0]*y[2])
#Si no se emite un spot en alguna de las franjas 4 o 5, tampoco se puede emitir en la franja 6.
#p4.add_constraint(y[3]*y[4]>=y[5])
p4.set_objective(p4.sum(Datos[j][1]*y[j] for j in range(len(Datos))))
print ('Valor objetivo=', p4.solve())
sol=p4.get_values(y)
for key in sorted(sol.keys()):
    print ('y',key+1,'=',sol[key])

Valor objetivo= 1030000.0
y 1 = 0.0
y 2 = 0.0
y 3 = 1.0
y 4 = 1.0
y 5 = 1.0
y 6 = 1.0
y 7 = 0.0
y 8 = 1.0


SAGE muestra un error si el la condición se están multiplicando dos variables, es por ello que se han comentado las condiciones no permitidas aunque en el pdf puede comprobarse que estas condiciones serían válidas en un principio. Aún con ello la restricción que hemos podido añadir ha modificado la respuesta anterior puesto que la franja 7 ya no es usada y es sustituida por otras.

# Problema 5 Apartado b)

Vamos a resolver el problema mediante SAGE. Tendremos la cantidad de producto 'x' que será entera y positiva y el factor 'y' binario que determina si una máquina ha sido usada o no:

In [118]:
p = MixedIntegerLinearProgram()
x = p.new_variable(integer=True, nonnegative=True)
y = p.new_variable(binary=True)
#Si elegimos un valor mayor que 0 para alguna x, su respectiva y debe ser mayor que 0:
p.add_constraint(x[0]<=y[0]*5)
p.add_constraint(x[1]<=y[1]*10)
p.add_constraint(x[2]<=y[2]*6)
#Restricción de número total de horas:
p.add_constraint(30*x[0]+20*x[1]+40*x[2]<=200)
#Restricción de número total de kilos de materia prima:
p.add_constraint(4*x[0]+3*x[1]+4*x[2]<=30)
#Restricciones de cantidad máxima de unidades de cada tipo que pueden generarse:
p.add_constraint(x[0]<=5)
p.add_constraint(x[1]<=10)
p.add_constraint(x[2]<=6)
#Y marcamos al programa la función objetivo.
p.set_objective(60*x[0]+40*x[1]+70*x[2]-200*y[0]-150*y[1]-100*y[2])
p.show()

Maximization:
  60.0 x_0 -200.0 x_1 + 40.0 x_2 -150.0 x_3 + 70.0 x_4 -100.0 x_5 

Constraints:
  x_0 - 5.0 x_1 <= 0.0
  x_2 - 10.0 x_3 <= 0.0
  x_4 - 6.0 x_5 <= 0.0
  30.0 x_0 + 20.0 x_2 + 40.0 x_4 <= 200.0
  4.0 x_0 + 3.0 x_2 + 4.0 x_4 <= 30.0
  x_0 <= 5.0
  x_2 <= 10.0
  x_4 <= 6.0
Variables:
  x_0 is an integer variable (min=0.0, max=+oo)
  x_1 is a boolean variable (min=0.0, max=1.0)
  x_2 is an integer variable (min=0.0, max=+oo)
  x_3 is a boolean variable (min=0.0, max=1.0)
  x_4 is an integer variable (min=0.0, max=+oo)
  x_5 is a boolean variable (min=0.0, max=1.0)


In [122]:
sol = p.solve()
for i, v in sorted(p.get_values(x).items()):
     print('x_%s = %s' % (i+1, v))
print("valor alcanzado:",sol)  

for i, v in sorted(p.get_values(y).items()):
     print('y_%s = %s' % (i+1, v))
  

x_1 = 0.0
x_2 = 10.0
x_3 = 0.0
valor alcanzado: 250.0
y_1 = 0.0
y_2 = 1.0
y_3 = 0.0


Como podemos observar se generarán 10 unidades del producto B y habrá un beneficio total de 250€. De forma concordante la única 'y' con valor 1 es y2, que relaciona al producto B e indica que se ha usado el tipo de máquina necesaria para generar B. No se han usado máquinas para generar producto A ó C.

# Problema 7 Apartado a) y b)

Vamos a resolver el problemas mediante SAGE:

In [131]:
p = MixedIntegerLinearProgram(maximization=False)
x = p.new_variable(binary=True)
y = p.new_variable(binary=True)

#Restricciones de tiempo 5 horas = 300 minutos
p.add_constraint(45*x[0] + 60*x[1] + 60*x[2] + 80*x[3] + 70*x[4] + 80*x[5] + 90*x[6] + 100*x[7]<=300)
p.add_constraint(45*y[0] + 60*y[1] + 60*y[2] + 80*y[3] + 70*y[4] + 80*y[5] + 90*y[6] + 100*y[7]<=300)
#Restricciones para que no coincida el mismo curso 2 días
p.add_constraint(x[0]+y[0]>=1)
p.add_constraint(x[1]+y[1]>=1)
p.add_constraint(x[2]+y[2]>=1)
p.add_constraint(x[3]+y[3]>=1)
p.add_constraint(x[4]+y[4]>=1)
p.add_constraint(x[5]+y[5]>=1)
p.add_constraint(x[6]+y[6]>=1)
p.add_constraint(x[7]+y[7]>=1)

#Y marcamos al programa la función objetivo.
p.set_objective((45*x[0] + 60*x[1] + 60*x[2] + 80*x[3] + 70*x[4] + 80*x[5] + 90*x[6] + 100*x[7])*200/60 + (45*y[0] + 60*y[1] + 60*y[2] + 80*y[3] + 70*y[4] + 80*y[5] + 90*y[6] + 100*y[7])*250/60)

p.show()

Minimization:
  150.0 x_0 + 200.0 x_1 + 200.0 x_2 + 266.6666666666667 x_3 + 233.33333333333334 x_4 + 266.6666666666667 x_5 + 300.0 x_6 + 333.3333333333333 x_7 + 187.5 x_8 + 250.0 x_9 + 250.0 x_10 + 333.3333333333333 x_11 + 291.6666666666667 x_12 + 333.3333333333333 x_13 + 375.0 x_14 + 416.6666666666667 x_15 

Constraints:
  45.0 x_0 + 60.0 x_1 + 60.0 x_2 + 80.0 x_3 + 70.0 x_4 + 80.0 x_5 + 90.0 x_6 + 100.0 x_7 <= 300.0
  45.0 x_8 + 60.0 x_9 + 60.0 x_10 + 80.0 x_11 + 70.0 x_12 + 80.0 x_13 + 90.0 x_14 + 100.0 x_15 <= 300.0
  - x_0 - x_8 <= -1.0
  - x_1 - x_9 <= -1.0
  - x_2 - x_10 <= -1.0
  - x_3 - x_11 <= -1.0
  - x_4 - x_12 <= -1.0
  - x_5 - x_13 <= -1.0
  - x_6 - x_14 <= -1.0
  - x_7 - x_15 <= -1.0
Variables:
  x_0 is a boolean variable (min=0.0, max=1.0)
  x_1 is a boolean variable (min=0.0, max=1.0)
  x_2 is a boolean variable (min=0.0, max=1.0)
  x_3 is a boolean variable (min=0.0, max=1.0)
  x_4 is a boolean variable (min=0.0, max=1.0)
  x_5 is a boolean variable (min=0.0, max=1.0)

In [133]:
sol = p.solve()
for i, v in sorted(p.get_values(x).items()):
     print('x_%s = %s' % (i+1, v))

for i, v in sorted(p.get_values(y).items()):
     print('y_%s = %s' % (i+1, v))
print("valor alcanzado:",sol)

x_1 = 0.0
x_2 = 1.0
x_3 = 1.0
x_4 = 1.0
x_5 = 0.0
x_6 = 0.0
x_7 = 0.0
x_8 = 1.0
y_1 = 1.0
y_2 = 0.0
y_3 = 0.0
y_4 = 0.0
y_5 = 1.0
y_6 = 1.0
y_7 = 1.0
y_8 = 0.0
valor alcanzado: 2187.5


Como podemos observar tendremos los cursos 2,3,4 y 8 el sábado. Los cursos 1,5,6 y 7 el domingo. El coste será de 2187.5€

Ahora añadamos las restricciones del siguiente apartado:

In [134]:
p = MixedIntegerLinearProgram(maximization=False)
x = p.new_variable(binary=True)
y = p.new_variable(binary=True)

p.add_constraint(45*x[0] + 60*x[1] + 60*x[2] + 80*x[3] + 70*x[4] + 80*x[5] + 90*x[6] + 100*x[7]<=300)
p.add_constraint(45*y[0] + 60*y[1] + 60*y[2] + 80*y[3] + 70*y[4] + 80*y[5] + 90*y[6] + 100*y[7]<=300)
p.add_constraint(x[0]+y[0]>=1)
p.add_constraint(x[1]+y[1]>=1)
p.add_constraint(x[2]+y[2]>=1)
p.add_constraint(x[3]+y[3]>=1)
p.add_constraint(x[4]+y[4]>=1)
p.add_constraint(x[5]+y[5]>=1)
p.add_constraint(x[6]+y[6]>=1)
p.add_constraint(x[7]+y[7]>=1)
#Cada día debe tener exactamente dos talleres de software.
p.add_constraint(x[0]+x[1]+x[3]+x[7]>=2)
p.add_constraint(y[0]+y[1]+y[3]+y[7]>=2)
#El sábado debe tener al menos un taller de hardware, y ademas contener alguno de los talleres entre software y ciberseguridad.
p.add_constraint(x[3]+x[4]+x[5]+x[7]>=1)
p.add_constraint(x[0]+x[1]+x[2]+x[6]+x[7]>=1)

#De nuevo SAGE da problemas si en la condición se multiplican variables.
#Si los talleres 2, 4 y 5 están en uno de los días, los talleres 4 y 8 deben estar en el otro día.
#p.add_constraint(x[1]+x[3]+x[4]<=3+(x[3]*x[7])-x[3]-x[7])

#Y marcamos al programa la función objetivo.
p.set_objective((45*x[0] + 60*x[1] + 60*x[2] + 80*x[3] + 70*x[4] + 80*x[5] + 90*x[6] + 100*x[7])*200/60 + (45*y[0] + 60*y[1] + 60*y[2] + 80*y[3] + 70*y[4] + 80*y[5] + 90*y[6] + 100*y[7])*250/60)

p.show()

Minimization:
  150.0 x_0 + 200.0 x_1 + 200.0 x_2 + 266.6666666666667 x_3 + 233.33333333333334 x_4 + 266.6666666666667 x_5 + 300.0 x_6 + 333.3333333333333 x_7 + 187.5 x_8 + 250.0 x_9 + 250.0 x_10 + 333.3333333333333 x_11 + 291.6666666666667 x_12 + 333.3333333333333 x_13 + 375.0 x_14 + 416.6666666666667 x_15 

Constraints:
  45.0 x_0 + 60.0 x_1 + 60.0 x_2 + 80.0 x_3 + 70.0 x_4 + 80.0 x_5 + 90.0 x_6 + 100.0 x_7 <= 300.0
  45.0 x_8 + 60.0 x_9 + 60.0 x_10 + 80.0 x_11 + 70.0 x_12 + 80.0 x_13 + 90.0 x_14 + 100.0 x_15 <= 300.0
  - x_0 - x_8 <= -1.0
  - x_1 - x_9 <= -1.0
  - x_2 - x_10 <= -1.0
  - x_3 - x_11 <= -1.0
  - x_4 - x_12 <= -1.0
  - x_5 - x_13 <= -1.0
  - x_6 - x_14 <= -1.0
  - x_7 - x_15 <= -1.0
  - x_0 - x_1 - x_3 - x_7 <= -2.0
  - x_8 - x_9 - x_11 - x_15 <= -2.0
  - x_3 - x_4 - x_5 - x_7 <= -1.0
  - x_0 - x_1 - x_2 - x_6 - x_7 <= -1.0
Variables:
  x_0 is a boolean variable (min=0.0, max=1.0)
  x_1 is a boolean variable (min=0.0, max=1.0)
  x_2 is a boolean variable (min=0.0, max=1

In [135]:
sol = p.solve()
for i, v in sorted(p.get_values(x).items()):
     print('x_%s = %s' % (i+1, v))

for i, v in sorted(p.get_values(y).items()):
     print('y_%s = %s' % (i+1, v))
print("valor alcanzado:",sol)

x_1 = 0.0
x_2 = 1.0
x_3 = 1.0
x_4 = 0.0
x_5 = 0.0
x_6 = 1.0
x_7 = 0.0
x_8 = 1.0
y_1 = 1.0
y_2 = 0.0
y_3 = 0.0
y_4 = 1.0
y_5 = 1.0
y_6 = 0.0
y_7 = 1.0
y_8 = 0.0
valor alcanzado: 2187.5


Como podemos observar con las nuevas restricciones tendremos los cursos 2,3,6 y 8 el sábado. Los cursos 1,4,5 y 7 el domingo. El coste será de 2187.5€